In [25]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from scipy import stats

# Data preparation

In [5]:
tot_df=pd.read_csv('25100_dat.csv')

In [6]:
def feature_transform(tot_df):
    corpus_meds=tot_df['meds_str'].values
    corpus_reacts=tot_df['reacts_str'].values
    
    c_med = CountVectorizer()
    med_feats = c_med.fit_transform(corpus_meds).toarray()
    
    c_r=CountVectorizer()
    reacts_feats=c_r.fit_transform(corpus_reacts).toarray()
    
    feats=np.concatenate([med_feats,reacts_feats],axis=1)
    
    return feats,c_med.vocabulary_, c_r.vocabulary_
    
feats,med_vocab,reacts_vocab=feature_transform(tot_df)

In [7]:
targets=tot_df[['death']].fillna(0)
print(targets)
np.mean(targets)
print(targets[targets.death==1].shape)

       death
0        0.0
1        1.0
2        0.0
3        0.0
4        0.0
...      ...
25095    0.0
25096    0.0
25097    0.0
25098    0.0
25099    0.0

[25100 rows x 1 columns]
(2162, 1)


In [8]:
# Separate majority and minority classes
df_majority = targets[targets.death==0]
df_minority = targets[targets.death==1]
 
#downsample
downsampled = resample(df_majority, 
                                 replace=True,     # sample with replacement
                                 n_samples=2162,    # to match majority class
                                 random_state=123) 

In [9]:
print(downsampled[downsampled.death==0])
downsampled_inds=downsampled[downsampled.death==0].index
death_inds=df_minority.index
print(downsampled_inds)
print(death_inds)

       death
21779    0.0
17071    0.0
19281    0.0
23595    0.0
16713    0.0
...      ...
5936     0.0
7182     0.0
20052    0.0
18979    0.0
11709    0.0

[2162 rows x 1 columns]
Int64Index([21779, 17071, 19281, 23595, 16713,  8326, 14528, 24302,   100,
            24751,
            ...
            19620, 21122,  5357, 22892,  6362,  5936,  7182, 20052, 18979,
            11709],
           dtype='int64', length=2162)
Int64Index([    1,    16,    19,    37,   209,   231,   244,   262,   269,
              272,
            ...
            24988, 24995, 25002, 25009, 25014, 25015, 25020, 25035, 25053,
            25074],
           dtype='int64', length=2162)


In [10]:
feats_no_death=feats[downsampled_inds,:]
feats_death=feats[death_inds,:]
feats=np.concatenate([feats_no_death,feats_death],axis=0)

In [11]:
targets=np.concatenate([downsampled.values,df_minority.values],axis=0)
print(targets.shape)


(4324, 1)


In [91]:
X_train, X_test, y_train, y_test = train_test_split(feats,targets,test_size=0.33)

# Logistic Regression

In [140]:
pcs=PCA(n_components=1000)
X_train_pcs=pcs.fit_transform(X_train)
X_test_pcs=pcs.transform(X_test)
print('Explained variance: ',pcs.explained_variance_ratio_.sum())

Explained variance:  0.9570552004058386


In [141]:
clf = LogisticRegression(random_state=0).fit(X_train_pcs, y_train)


/home/maria/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [142]:
pred=clf.predict(X_test_pcs)
accuracy_score(pred,y_test)

0.8857743517869656

In [143]:
confusion_matrix(y_test,pred,labels=[1,0])

array([[624, 101],
       [ 62, 640]])

In [251]:
r2_score(y_test, pred)

0.44484526967285576

# Naive Bayes

In [145]:
clf=MultinomialNB().fit(X_train,y_train)

/home/maria/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [231]:
pred=clf.predict(X_test)
accuracy_score(pred,y_test)

0.8612473721093202

In [232]:
confusion_matrix(y_test,pred,labels=[1,0])

array([[662,  63],
       [135, 567]])

In [219]:
#print(med_vocab)
#print(reacts_vocab)

tot_vocab={}
for j in med_vocab.keys():
    tot_vocab[med_vocab[j]]=j
for i in reacts_vocab.keys():
    tot_vocab[reacts_vocab[i]+len(med_vocab.keys())]=i
print(tot_vocab)

{6210: 'pyostacine', 6065: 'pristinamycin', 5558: 'oroken', 1814: 'cefixime', 7537: 'unspec', 6482: 'rocephin', 5443: 'ofloxacin', 2754: 'duragesic', 347: '100', 1501: 'boniva', 3847: 'ibuprofen', 4600: 'lyrica', 3170: 'fiorinal', 4534: 'lortab', 2101: 'codeine', 4028: 'iodine', 7347: 'tramadol', 3777: 'hydrochloride', 1125: 'aspirin', 5620: 'oxycontin', 7050: 'talwin', 2707: 'doxycycline', 3760: 'hyclate', 4440: 'lipitor', 4537: 'losartan', 5959: 'potassium', 4851: 'metoprolol', 7083: 'tartrate', 4460: 'lisinopril', 1269: 'bactrim', 4923: 'minocycline', 4338: 'letairis', 7491: 'tyvaso', 5622: 'oxygen', 5246: 'nexium', 6857: 'spiriva', 673: 'adcirca', 7927: 'zocor', 7808: 'xanax', 7395: 'triamterene', 709: 'advair', 3710: 'hfa', 7336: 'tracleer', 7510: 'ultracet', 6348: 'relpax', 867: 'ambien', 3874: 'ilaris', 6005: 'prednisone', 6280: 'ranolazine', 5897: 'plaquenil', 1344: 'benlysta', 1666: 'calcium', 1742: 'carbonate', 2058: 'clonazepam', 1321: 'belimumab', 7721: 'vitamin', 1234: 'az

In [248]:
print(clf.feature_log_prob_.shape)
print(len(tot_vocab.keys()))
probs=clf.feature_log_prob_
top_death_class=probs[1,:].argsort()[-100:][::-1]
print(probs[0,top_death_class])
for j in top:
    print(tot_vocab[j])

(2, 11094)
11094
[ -8.31127542  -6.77083038  -7.90581031  -6.34183477  -6.96754067
  -5.59217538  -5.92864762  -6.66904769  -6.05998362  -4.51953858
  -7.33044617  -7.01199244  -4.7703161   -8.49359698  -5.36683644
  -7.46397756  -6.0957017   -7.15859591  -6.84493835  -5.67221809
  -7.15859591  -8.15712474  -6.66904769  -8.15712474  -6.96754067
  -5.92864762  -8.31127542  -6.84493835  -8.31127542  -6.96754067
  -8.31127542  -7.10730262  -6.38946282  -8.02359335  -6.15179117
  -6.73573906  -6.31884526  -6.0957017   -6.63729899  -7.26982155
  -6.15179117  -7.33044617  -9.40988771  -6.2963724   -8.02359335
  -7.90581031  -7.90581031  -8.31127542  -6.38946282  -5.85453965
  -7.21266313  -7.90581031  -8.49359698  -7.26982155  -7.26982155
  -8.15712474  -8.02359335  -8.02359335  -5.62569808  -8.02359335
  -8.02359335  -7.90581031  -6.63729899  -6.88415907  -9.0044226
  -8.71674053  -9.0044226   -6.70183751  -6.73573906  -8.31127542
  -7.53808553  -7.61812824  -8.49359698  -8.31127542  -8.311

# K-fold crossvalidation

In [15]:
kf = KFold(n_splits=10,shuffle=True)

acc_lst_lr=[]
acc_lst_nb=[]
for train_index, test_index in kf.split(feats):
    
    X_train, X_test = feats[train_index], feats[test_index]
    y_train, y_test = targets[train_index].reshape(-1,), targets[test_index].reshape(-1,)
    pcs=PCA(n_components=1000)
    X_train_pcs=pcs.fit_transform(X_train)
    X_test_pcs=pcs.transform(X_test)
    clf = LogisticRegression(random_state=0).fit(X_train_pcs, y_train)
    pred=clf.predict(X_test_pcs)
    acc_lst_lr.append(accuracy_score(pred,y_test))
    clf=MultinomialNB().fit(X_train,y_train)
    pred=clf.predict(X_test)
    acc_lst_nb.append(accuracy_score(pred,y_test))

print('Logistic regression k-fold accuracies:', acc_lst_lr)
print('Naive Bayes k-fold accuracies:', acc_lst_nb)
    

Logistic regression k-fold accuracies: [0.9099307159353349, 0.9145496535796767, 0.8891454965357968, 0.8614318706697459, 0.8680555555555556, 0.8912037037037037, 0.9027777777777778, 0.9120370370370371, 0.9027777777777778, 0.8865740740740741]
Naive Bayes k-fold accuracies: [0.8822170900692841, 0.8637413394919169, 0.8475750577367206, 0.8498845265588915, 0.8518518518518519, 0.8611111111111112, 0.8634259259259259, 0.8773148148148148, 0.8634259259259259, 0.8773148148148148]


In [24]:
print('Logistic regression accuracy 95% confidence interval:', np.mean(acc_lst_lr)-1.654*(np.std(acc_lst_lr)/np.sqrt(10)),np.mean(acc_lst_lr)+1.654*(np.std(acc_lst_lr)/np.sqrt(10)))
print('Naive Bayes accuracy 95% confidence interval:', np.mean(acc_lst_nb)-1.654*(np.std(acc_lst_nb)/np.sqrt(10)),np.mean(acc_lst_nb)+1.654*(np.std(acc_lst_nb)/np.sqrt(10)))

Logistic regression accuracy 95% confidence interval: 0.8848259416337891 0.9028707908955071
Naive Bayes accuracy 95% confidence interval: 0.8577960054336116 0.8697764862266397


In [27]:
var_lr = np.array(acc_lst_lr).var(ddof=1)
var_nb = np.array(acc_lst_nb).var(ddof=1)
#std deviation
s = np.sqrt((var_lr + var_nb)/2)
## Calculate the t-statistics
t = (np.array(acc_lst_lr).mean() - np.array(acc_lst_nb).mean())/(s*np.sqrt(2/N))



## Compare with the critical t-value
#Degrees of freedom
df = 2*N - 2

#p-value after comparison with the t 
p = 1 - stats.t.cdf(t,df=df)